# Thompson Sampling on Bernoulli

In [3]:
import numpy as np

In [4]:
NUM_OF_BANDITS = 10
NUM_OF_TRIALS = 5000

In [21]:
class TS_Bandit:
    def __init__(self, probability):
        # For log purposes
        self.result = []
        
        # latent variables
        self.probability = probability
        
        # Bandit will have the property of beta distribution as the conjugate prior of the bernoulli distribution
        self.alfa = 0
        self.beta = 0        
    
    def update_posterior(self, reward):
        if reward:
            self.alfa += 1
        else:
            self.beta += 1
    
    def pull_bandit_arm(self):
        # Generate random bernoulli as reward
        # parameter:
        # p = success probability
        # f(x) = p^x * (1-p)^1-x
        reward = np.random.binomial(n=1, p=self.probability, size=None)
        
        self.update_posterior(reward)
        
        return reward
        

In [52]:
temp_bandit = TS_Bandit(0.8)

In [62]:
for i in range (10000):
    reward = temp_bandit.pull_bandit_arm()

print(reward)
print(f'{temp_bandit.alfa} || {temp_bandit.beta}')

0
80165 || 19835
